<style>div.title-slide {    width: 100%;    display: flex;    flex-direction: row;            /* default value; can be omitted */    flex-wrap: nowrap;              /* default value; can be omitted */    justify-content: space-between;}</style><div class="title-slide">
<span style="float:left;">Licence CC BY-NC-ND</span>
<span>Thierry Parmentelat &amp; Arnaud Legout</span>
<span><img src="media/both-logos-small-alpha.png" style="display:inline" /></span>
</div>

# *Context managers* et exceptions

## Complément - niveau intermédiaire

On a vu jusqu'ici dans la vidéo comment écrire un context manager, mais on n'a pas envisagé le cas où une exception serait levée pendant la durée de vie du context manager.

Et c'est très important, car si je me contente de faire :

In [ ]:
import time

class Timer1:
    def __enter__(self):
        print("Entering Timer1")
        self.start = time.time()
        return self
    
    def __exit__(self, *args):
        print(f"Total duration {time.time()-self.start:2f}")
        return True

Alors dans les cas nominaux, tout se passe comme attendu :

In [ ]:
with Timer1():
    n = 0
    for i in range(2*10**6):
        n += i**2

Mais par contre, dans le cas où j'exécute du code qui lève une exception, ça ne va plus du tout :

In [ ]:
with Timer1():
    n = 0
    for i in range(2*10**6):
        n += i**2 / 0

À la toute première itération de la boucle, on fait une division par 0, qui lève l'exception `ZeroDivisionError`, mais tel qu'est conçue notre classe de context manager, cette exception **est étouffée** et n'est pas correctement propagée à l'extérieur.

Il est important, lorsqu'on conçoit un context manager, de bien **propager** les exceptions qui ne sont pas liées au fonctionnement attendu du context manager. Par exemple un objet de type fichier va en effet attraper par exemple les exceptions liées à la fin du fichier, mais doit par contre laisser passer une exception comme `ZeroDivisionError`.

### Les paramètres de `__exit__`

Comme [vous pouvez le retrouver ici](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers), la méthode `__exit__` reçoit trois arguments :

    def __exit__(self, exc_type, exc_value, traceback):

lorsqu'on sort du bloc `with` sans qu'une exception soit levée, ces trois arguments valent `None`. Par contre si une exception est levée, ils permettent d'accéder au type, à la valeur de l'exception, et à l'état de la pile lorsque l'exception est levée.

In [ ]:
# une deuxième version de Timer
# qui propage correctement les exceptions

class Timer2:
    def __enter__(self):
        print("Entering Timer1")
        self.start = time.time()
        # rappel : le retour de __enter__ est ce qui est passé
        # à la clause `as` du `with`
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        if exc_type is None:
            print(f"Total duration {time.time()-self.start:2f}")
            # ceci indique que tout s'est bien passé
            return True
        else:
            print(f"OOPS : on propage l'exception {exc_type} - {exc_value}")
            # c'est ici que je propage l'exception au dehors du with
            raise exc_type(exc_value)
        return True

In [ ]:
try:
    with Timer2():
        n = 0
        for i in range(2*10**6):
            n += i**2 / 0
except Exception as e:
    print(f"L'exception a bien été propagée, {type(e)} - {e}")

### Pour en savoir plus

Je vous signale enfin [la bibliothèque `contextlib`](https://docs.python.org/3/library/contextlib.html) qui offre quelques utilitaires pour se définir un contextmanager.

Notamment, un peu comme on peut implémenter un itérateur comme un générateur qui fait (n'importe quel nombre de) `yield`, on peut également implémenter un context manager simple sous la forme d'une fonction qui fait un `yield`.

xxx
un étudiant relève ceci:

Bonjour,

Je ne saisis pas trop pourquoi, dans le code présenté en `w6-s8-c1` où l'on construit un Context Manager, on renvoie toujours `True` ou que l'on re-lève l'exception s'il y en a eu une.

[La documentation](https://docs.python.org/3/reference/datamodel.html#with-statement-context-managers), vers laquelle vous pointez, décrit un protocole plus simple et qui me semble plus naturel :

> If an exception is supplied, and the method wishes to suppress the exception (i.e., prevent it from being propagated), it should return a true value. Otherwise, the exception will be processed normally upon exit from this method.
> 
> Note that __exit__() methods should not reraise the passed-in exception; this is the caller’s responsibility.

Dit autrement: il me semble que dans le premier code présenté, il n'y a rien d'autre à faire d'autre que d'omettre le `return True` en fin de méthode `__exit__()`, et tout l'exercice fonctionnerait sans se poser plus de questions...

N'aurait-il pas fallu, pour exposer l'utilité du `return True`, utiliser un exemple où l'on veut justement empêcher la remontée de l'exception ?

Merci d'avance pour votre réponse, et bonne année !
